# X-ray absorption spectroscopy (XAS)

## Import the required packages and set up logging

In [ ]:
%matplotlib widget

import sys
import logging

import numpy as np
import matplotlib.pyplot as plt

from daxs.measurements import Hdf5Source, Xas

logging.basicConfig(level=logging.INFO, stream=sys.stdout)
logging.getLogger("daxs").setLevel(logging.INFO)

## Locate the experimental data using operating system commands

In [ ]:
!ls experimental_data

## Define the data source and create the measurement

In [ ]:
filename = "experimental_data/ihch1515/id26/Cr2O3_new/Cr2O3_new_0002/Cr2O3_new_0002.h5"
included_scans = "4-23"
data_mappings = {"x": ".1/measurement/hdh_energy", "signal": ".1/measurement/det_dtc_apd", "monitor": ".1/measurement/I02"}

source = Hdf5Source(filename, included_scans, data_mappings=data_mappings)
measurement = Xas(source)

## Plot the reduced data

In [ ]:
fig, ax = plt.subplots()

ax.plot(measurement.x, measurement.signal)

ax.set_xlabel("Incident energy (keV)")
ax.set_ylabel("Intensity (arb. units)")

plt.tight_layout()

## Aggregation of multiple scans

Fraction of sums:
- $I_S = \frac{I_{S,1}(E)\ +\ I_{S,2}(E)\ +\ \cdots}{I_{M,1}(E)\ +\ I_{M,2}(E)\ +\ \cdots}$ 
- This may not correct well for instabilities in $I_M$.
   
Sum of fractions:
- $I_S = \frac{I_{S,1}(E)}{I_{M,1}(E)} + \frac{I_{S,2}(E)}{I_{M,2}(E)} + \cdots$ 
- This may not give the correct statistical weight between scans and lose the total counts.

In [ ]:
fig, ax = plt.subplots()

ax.plot(measurement.x, measurement.signal, label="fraction of sums")

measurement.reset()
measurement.process(aggregation="sum of fractions")
ax.plot(measurement.x, measurement.signal, label="sum of fractions")

ax.legend()
ax.set_xlabel("Incident energy (keV)")
ax.set_ylabel("Intensity (arb. units)")

plt.tight_layout()

## Normalize the reduced data

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, gridspec_kw={"height_ratios": [3, 1]})

measurement.reset()
measurement.process(aggregation="fraction of sums", normalization="maximum")
ax1.plot(measurement.x, measurement.signal, label="fraction of sums")

# Save the data for the calculation of the difference signal.
x = np.copy(measurement.x)
signal_fos = np.copy(measurement.signal)

measurement.reset()
measurement.process(aggregation="sum of fractions", normalization="maximum")
ax1.plot(measurement.x, measurement.signal, label="sum of fractions")

# Save the data for the calculation of the difference signal.
signal_sof = np.copy(measurement.signal)

ax1.legend()
ax1.set_xlabel("Incident energy (keV)")
ax1.set_ylabel("Intensity (arb. units)")

# Plot the difference of the two signals.
ax2.plot(x, signal_fos - signal_sof)

plt.tight_layout()